In [62]:
import MetaTrader5 as mt5
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
warnings.filterwarnings("ignore")
mt5.initialize()
account=51127988
password="Aar2frM7"
server = 'ICMarkets-Demo'

from collections import defaultdict
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

def get_rates(pair1, tf, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, tf, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    pair1 = pair1.set_index(pair1['time'])
    pair1 = pair1.drop(columns = ['time','tick_volume', 'spread', 'real_volume'])
    return pair1

# Assume mini_list and get_rates are defined elsewhere.
mini_list = ['AUDUSD.a', 'CADJPY.a', 'EURCAD.a', 'EURGBP.a']

def buy_order(symbol):
    price = mt5.symbol_info_tick(symbol).ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 1.00,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": f"Rips Complex",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result1 = mt5.order_send(request)
    result1
    
def sell_order(symbol):
    price = mt5.symbol_info_tick(symbol).bid
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 1.00,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": f"Rips Complex",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result1 = mt5.order_send(request)
    result1

def t3_sing_corr(symbol, base_hour, bars=100):
    base_ts1 = datetime.strptime(f'{base_hour}:00', '%H:%M').time()
    base_ts2 = (datetime.combine(datetime.today(), base_ts1) + timedelta(hours=1)).time()
    base_ts1, base_ts2 = [t.strftime('%H:%M') for t in [base_ts1, base_ts2]]

    base_prices = get_rates(symbol, mt5.TIMEFRAME_H1, 50000)
    base_prices = base_prices.between_time(base_ts1, base_ts2)
    base_prices['returns'] = base_prices['close'] - base_prices['open']

    correlations = {}
    ordered_correlations = defaultdict(list)

    for h in range(24):
        if h == base_hour:
            continue

        ts1 = datetime.strptime(f'{h}:00', '%H:%M').time()
        ts2 = (datetime.combine(datetime.today(), ts1) + timedelta(hours=1)).time()
        ts1, ts2 = [t.strftime('%H:%M') for t in [ts1, ts2]]

        prices = get_rates(symbol, mt5.TIMEFRAME_H1, 50000)
        prices = prices.between_time(ts1, ts2)
        prices['returns'] = prices['close'] - prices['open']

        pearson_corr = np.corrcoef(base_prices['returns'].tail(bars), prices['returns'].tail(bars))[0, 1]

        if abs(pearson_corr) > 0.2:
            correlations[f'{base_ts1}-{ts1} | {symbol}'] = pearson_corr

        if abs(pearson_corr) > 0.4:
            ordered_correlations['0.4 and over'].append((f'{base_ts1}-{ts1}', pearson_corr, symbol))
        elif abs(pearson_corr) > 0.3:
            ordered_correlations['0.3 to 0.4'].append((f'{base_ts1}-{ts1}', pearson_corr, symbol))
        elif abs(pearson_corr) > 0.2:
            ordered_correlations['0.2 to 0.3'].append((f'{base_ts1}-{ts1}', pearson_corr, symbol))

    return correlations, ordered_correlations

In [63]:
correlations_all = {}
ordered_correlations_all = defaultdict(list)

for symbol in mini_list:
    for i in range(24):
        correlations, ordered_correlations = t3_sing_corr(symbol, i, bars=100)
        correlations_all.update(correlations)
        for key in ordered_correlations.keys():
            ordered_correlations_all[key].extend(ordered_correlations[key])

print('Analysis Complete')

data = ordered_correlations_all
ordered_correlations_all = defaultdict(list)
for key in data.keys():
    ordered_correlations_all[key] = [item for item in data[key] if int(item[0].split('-')[0].split(':')[0]) < int(item[0].split('-')[1].split(':')[0])]

strong_correlations = ordered_correlations_all['0.4 and over']
correct_data = [item for item in strong_correlations if int(item[0].split('-')[0].split(':')[0]) < int(item[0].split('-')[1].split(':')[0])]
print('Sorting complete. Data:')
print(correct_data)

Analysis Complete
Sorting complete. Data:
[('11:00-12:00', 0.5381163921667252, 'AUDUSD.a'), ('12:00-13:00', 0.44225193676652935, 'AUDUSD.a'), ('06:00-19:00', 0.5256722522789767, 'CADJPY.a'), ('07:00-20:00', 0.4442479992818048, 'CADJPY.a'), ('07:00-22:00', 0.4828922502213256, 'CADJPY.a'), ('12:00-13:00', 0.5300800376933912, 'CADJPY.a'), ('20:00-23:00', -0.4340704345773939, 'CADJPY.a'), ('12:00-13:00', 0.5355769611195893, 'EURCAD.a'), ('12:00-13:00', 0.47166712092482116, 'EURGBP.a')]


# Logic of trade

The following timestamp of the two correlated times is the subject of interest because whatever return occured in its partnered pair has a high correlation in the return of the timestamp afterwards. 

1. Check if 'current_time' is equal to the latter timestamp of the correlated times
2. If it's equal:
    - Calculate return of the former timestamp for the symbol and confirm which direction to take (Same directiion as return)
    - Send trade order in calculated direction
    - Close trade at the end of the hour

In [57]:
now = datetime.now(pytz.timezone('Europe/Helsinki'))  # use the appropriate timezone
current_time = now.strftime('%H:%M')
# current_time2 = '20:00'
pairs2trade = []

for data in correct_data:
    time_range = data[0]
    start_time, end_time = [t.strftime('%H:%M') for t in [datetime.strptime(t, '%H:%M') for t in time_range.split('-')]]
    if end_time == current_time:
        print(f'Match Found for {data[2]} at {end_time}. Adding to list to trade...')
        pairs2trade.append(data)
        
    else:
        print(f'No Match on {data[2]}. Time is {current_time}. Correlated time is {end_time}')


No Match on AUDUSD.a. Time is 16:17. Correlated time is 11:00
No Match on AUDUSD.a. Time is 16:17. Correlated time is 12:00
No Match on CADJPY.a. Time is 16:17. Correlated time is 23:00
No Match on CADJPY.a. Time is 16:17. Correlated time is 19:00
No Match on CADJPY.a. Time is 16:17. Correlated time is 20:00
No Match on CADJPY.a. Time is 16:17. Correlated time is 22:00
No Match on CADJPY.a. Time is 16:17. Correlated time is 11:00
No Match on CADJPY.a. Time is 16:17. Correlated time is 23:00
No Match on EURCAD.a. Time is 16:17. Correlated time is 23:00
No Match on EURCAD.a. Time is 16:17. Correlated time is 11:00
No Match on EURGBP.a. Time is 16:17. Correlated time is 23:00
No Match on EURGBP.a. Time is 16:17. Correlated time is 11:00


[]

In [14]:
def get_rate(pair1, tf, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, tf, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    pair1 = pair1.set_index(pair1['time'])
    pair1 = pair1.drop(columns = ['time','tick_volume', 'spread', 'real_volume'])
    return pair1
# pairs2trade

In [19]:
from datetime import time
buy_list = []
sell_list = []

for pair in pairs2trade:
    rates = get_rate(pair[2], mt5.TIMEFRAME_H1, 24)
    rates.index = rates.index.time
    target_time = time(20,0,0)
    info = rates.loc[target_time]
    returns = (info['close'] - info['open'])
    print(f"Returns for {pair[2]} are {returns}")

    if returns > 0:
        print(f"Init. Buy for {pair}")
        buy_list.append(pair)
        
    else:
        print(f"Init. Sell for {pair}")
        sell_list.append(pair)
    
print(buy_list)
print(sell_list)

Returns for CADJPY.a are 0.0040000000000048885
Init. Buy for ('07:00-20:00', 0.4468491082964434, 'CADJPY.a')
[('07:00-20:00', 0.4468491082964434, 'CADJPY.a')]
[]


In [53]:
import datetime

# Store position opening times
open_positions = {}

# ... code to open position ...

# After opening a position, store the opening time
for position in mt5.positions_get():
    open_positions[position.ticket] = datetime.datetime.now()

# ... main loop ...

# Iterate through open positions
if len(open_positions) >= 1:
    print('Beginning closing logic')
    for ticket, open_time in open_positions.items():
        # Get the current time
        current_time = datetime.datetime.now()
        # Calculate the difference between the current time and the open time
        time_difference = current_time - open_time
        # If the position has been open for more than an hour
        if time_difference.total_seconds() > 3600:  # 3600 seconds = 1 hour
            # Close the position
            position = mt5.positions_get(ticket=ticket)[0]
            close_position(position)
            # Remove the position from the dictionary
            del open_positions[ticket]
else:
    print('No open positions')

Beginning closing logic


In [95]:
mt5.account_info()

AccountInfo(login=300733693, trade_mode=0, leverage=100, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=19022.66, credit=0.0, profit=0.0, equity=19022.66, margin=0.0, margin_free=19022.66, margin_level=0.0, margin_so_call=100.0, margin_so_so=100.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='My Forex Funds - Evaluation Phase 1 Demo - Mark Paje ', server='TradersGlobalGroup-Demo', currency='USD', company='Traders Global Group Incorporated')

In [96]:
def run():

    import MetaTrader5 as mt5
    import pandas as pd
    import numpy as np
    import warnings
    warnings.filterwarnings("ignore")
    
    account = 51127988
    password="Aar2frM7"
    server = 'ICMarkets-Demo'
    
    mt5.initialize()

    account = 51127988
    password="Aar2frM7"
    server = 'ICMarkets-Demo'
    
    print(mt5.account_info())

    from collections import defaultdict
    from datetime import datetime, timedelta

    def get_rates(pair1, tf, x):
        pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, tf, 0, x))
        pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
        pair1 = pair1.set_index(pair1['time'])
        pair1 = pair1.drop(columns = ['time','tick_volume', 'spread', 'real_volume'])
        return pair1

    # Assume mini_list and get_rates are defined elsewhere.
    mini_list = ['AUDUSD.a', 'CADJPY.a', 'EURCAD.a', 'EURGBP.a']

    def buy_order(symbol):
        price = mt5.symbol_info_tick(symbol).ask
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 1.00,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "deviation": 20,
            "magic": 234000,
            "comment": f"Rips Complex",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result1 = mt5.order_send(request)
        result1
        
    def sell_order(symbol):
        price = mt5.symbol_info_tick(symbol).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 1.00,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "deviation": 20,
            "magic": 234000,
            "comment": f"Rips Complex",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result1 = mt5.order_send(request)
        result1

    def t3_sing_corr(symbol, base_hour, bars=100):
        base_ts1 = datetime.strptime(f'{base_hour}:00', '%H:%M').time()
        base_ts2 = (datetime.combine(datetime.today(), base_ts1) + timedelta(hours=1)).time()
        base_ts1, base_ts2 = [t.strftime('%H:%M') for t in [base_ts1, base_ts2]]

        base_prices = get_rates(symbol, mt5.TIMEFRAME_H1, 50000)
        base_prices = base_prices.between_time(base_ts1, base_ts2)
        base_prices['returns'] = base_prices['close'] - base_prices['open']

        correlations = {}
        ordered_correlations = defaultdict(list)

        for h in range(24):
            if h == base_hour:
                continue

            ts1 = datetime.strptime(f'{h}:00', '%H:%M').time()
            ts2 = (datetime.combine(datetime.today(), ts1) + timedelta(hours=1)).time()
            ts1, ts2 = [t.strftime('%H:%M') for t in [ts1, ts2]]

            prices = get_rates(symbol, mt5.TIMEFRAME_H1, 50000)
            prices = prices.between_time(ts1, ts2)
            prices['returns'] = prices['close'] - prices['open']

            pearson_corr = np.corrcoef(base_prices['returns'].tail(bars), prices['returns'].tail(bars))[0, 1]

            if abs(pearson_corr) > 0.2:
                correlations[f'{base_ts1}-{ts1} | {symbol}'] = pearson_corr

            if abs(pearson_corr) > 0.4:
                ordered_correlations['0.4 and over'].append((f'{base_ts1}-{ts1}', pearson_corr, symbol))
            elif abs(pearson_corr) > 0.3:
                ordered_correlations['0.3 to 0.4'].append((f'{base_ts1}-{ts1}', pearson_corr, symbol))
            elif abs(pearson_corr) > 0.2:
                ordered_correlations['0.2 to 0.3'].append((f'{base_ts1}-{ts1}', pearson_corr, symbol))

        return correlations, ordered_correlations

    correlations_all = {}
    ordered_correlations_all = defaultdict(list)

    for symbol in mini_list:
        # print(symbol)
        for i in range(24):
            # print(symbol, i)
            correlations, ordered_correlations = t3_sing_corr(symbol, i, bars=100)
            correlations_all.update(correlations)
            for key in ordered_correlations.keys():
                ordered_correlations_all[key].extend(ordered_correlations[key])

    print('Analysis Complete')

    data = ordered_correlations_all
    ordered_correlations_all = defaultdict(list)
    for key in data.keys():
        ordered_correlations_all[key] = [item for item in data[key] if int(item[0].split('-')[0].split(':')[0]) < int(item[0].split('-')[1].split(':')[0])]

    strong_correlations = ordered_correlations_all['0.4 and over']
    correct_data = [item for item in strong_correlations if int(item[0].split('-')[0].split(':')[0]) < int(item[0].split('-')[1].split(':')[0])]
    print('Sorting complete. Data:')
    print(correct_data)

    now = datetime.now(pytz.timezone('Europe/Helsinki'))  # use the appropriate timezone
    current_time = now.strftime('%H:%M')
    # current_time2 = '20:00'
    pairs2trade = []

    for data in correct_data:
        time_range = data[0]
        start_time, end_time = [t.strftime('%H:%M') for t in [datetime.strptime(t, '%H:%M') for t in time_range.split('-')]]
        if end_time == current_time:
            print(f'Match Found for {data[2]} at {end_time}. Adding to list to trade...')
            pairs2trade.append(data)
            
        else:
            print(f'No Match on {data[2]}. Time is {current_time}. Correlated time is {end_time}')

    def get_rate(pair1, tf, x):
        pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, tf, 0, x))
        pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
        pair1 = pair1.set_index(pair1['time'])
        pair1 = pair1.drop(columns = ['time','tick_volume', 'spread', 'real_volume'])
        return pair1

    from datetime import time
    buy_list = []
    sell_list = []
    
    print('Entering trading loop')
    
    for pair in pairs2trade:
        print(pair)
        rates = get_rate(pair[2], mt5.TIMEFRAME_H1, 24)
        rates.index = rates.index.time
        target_time = time(20,0,0)
        info = rates.loc[target_time]
        returns = (info['close'] - info['open'])
        print(f"Returns for {pair[2]} are {returns}")

        if returns > 0:
            print(f"Init. Buy for {pair}")
            buy_list.append(pair)
            
        else:
            print(f"Init. Sell for {pair}")
            sell_list.append(pair)
        
    print(buy_list)
    print(sell_list)

    import datetime

    # Store position opening times
    open_positions = {}

    # ... code to open position ...

    # After opening a position, store the opening time
    for position in mt5.positions_get():
        open_positions[position.ticket] = datetime.datetime.now()

    # ... main loop ...

    # Iterate through open positions
    if len(open_positions) >= 1:
        print('Beginning closing logic')
        for ticket, open_time in open_positions.items():
            # Get the current time
            current_time = datetime.datetime.now()
            # Calculate the difference between the current time and the open time
            time_difference = current_time - open_time
            # If the position has been open for more than an hour
            if time_difference.total_seconds() > 3600:  # 3600 seconds = 1 hour
                # Close the position
                position = mt5.positions_get(ticket=ticket)[0]
                close_position(position)
                # Remove the position from the dictionary
                del open_positions[ticket]
    else:
        print('No open positions') 

In [100]:
run()

AccountInfo(login=300733693, trade_mode=0, leverage=100, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=19022.66, credit=0.0, profit=0.0, equity=19022.66, margin=0.0, margin_free=19022.66, margin_level=0.0, margin_so_call=100.0, margin_so_so=100.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='My Forex Funds - Evaluation Phase 1 Demo - Mark Paje ', server='TradersGlobalGroup-Demo', currency='USD', company='Traders Global Group Incorporated')


KeyError: 'time'

In [67]:
import time 

while True:
    run()
    
    print('Strategy Inter-lagged Loop Completed')
    
    time.sleep(30)

Analysis Complete
Sorting complete. Data:
[('14:00-15:00', 0.6478555314873491, 'AUDUSD.a'), ('06:00-19:00', 0.5256722522789767, 'CADJPY.a'), ('07:00-20:00', 0.4442479992818048, 'CADJPY.a'), ('07:00-22:00', 0.4828922502213256, 'CADJPY.a'), ('14:00-15:00', 0.6247276650106708, 'CADJPY.a'), ('20:00-23:00', -0.4340704345773939, 'CADJPY.a'), ('14:00-15:00', 0.729163056468036, 'EURCAD.a'), ('13:00-14:00', 0.6080780244139647, 'EURGBP.a'), ('14:00-15:00', 0.6022938329112788, 'EURGBP.a')]
No Match on AUDUSD.a. Time is 14:54. Correlated time is 15:00
No Match on CADJPY.a. Time is 14:54. Correlated time is 19:00
No Match on CADJPY.a. Time is 14:54. Correlated time is 20:00
No Match on CADJPY.a. Time is 14:54. Correlated time is 22:00
No Match on CADJPY.a. Time is 14:54. Correlated time is 15:00
No Match on CADJPY.a. Time is 14:54. Correlated time is 23:00
No Match on EURCAD.a. Time is 14:54. Correlated time is 15:00
No Match on EURGBP.a. Time is 14:54. Correlated time is 14:00
No Match on EURGBP.a

KeyError: 'time'

In [42]:
# Calculate daily returns
eu_daily['return'] = eu_daily['close'].pct_change()

# Calculate hourly returns
eu_hrly['return'] = eu_hrly['close'].pct_change()

correlations = []

for index, row in eu_daily.iterrows():
    day = row['time']
    daily_return = row['return']
    
    # Select the 24 hourly data points after this day
    mask = (eu_hrly['time'] > day) & (eu_hrly['time'] <= (pd.to_datetime(day) + pd.Timedelta(days=1)))
    eu_hrly = eu_hrly.loc[mask]
    
    for _, h_row in eu_hrly.iterrows():
        eu_hrly = h_row['return']
        correlation = eu_daily * eu_hrly  # Calculate correlation (using a simple multiplication here for illustration)
        correlations.append(correlation)

# average_correlation = sum(correlations) / len(correlations)
# print(average_correlation)

TypeError: cannot perform __mul__ with this index type: DatetimeArray